In [1]:
from matplotlib import pyplot
from math import pi
import numpy
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
import numba
from numba import jit

In [2]:
def uex(t,X,Y):
    return -numpy.exp(-2*t)*numpy.cos(X)*numpy.sin(Y)

def vex(t,X,Y):
    return numpy.exp(-2*t)*numpy.sin(X)*numpy.cos(Y)

def pex(t,X,Y):
    return -numpy.exp(-4*t)/4.*(numpy.cos(2*X)+numpy.cos(2*Y))

In [3]:
def Fc1(u,v,dx,dy):
    F=numpy.zeros(numpy.shape(u))
    F[1:-1,1:-1]=-((u[1:-1,1:-1]+u[1:-1,2:])**2-(u[1:-1,1:-1]+u[1:-1,:-2])**2)/(4*dx)-\
                ((u[1:-1,1:-1]+u[2:,1:-1])*(v[1:-1,2:]+v[1:-1,1:-1])-(u[1:-1,1:-1]+u[:-2,1:-1])*(v[:-2,2:]+v[:-2,1:-1]))/(4*dy)
        
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [4]:
def Fv1(u,v,dx,dy):
    F=numpy.zeros(numpy.shape(u))
    F[1:-1,1:-1]=(u[1:-1,2:]-2*u[1:-1,1:-1]+u[1:-1,:-2])/dx**2+(u[2:,1:-1]-2*u[1:-1,1:-1]+u[:-2,1:-1])/dy**2
    
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [5]:
def Fp1(p,dx,dy):
    F=numpy.zeros(numpy.shape(p))
    F[1:-1,1:-1]=-(p[1:-1,2:]-p[1:-1,1:-1])/dx
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [6]:
def Fc2(u,v,dx,dy):
    F=numpy.zeros((numpy.shape(u)))
    F[1:-1,1:-1]=-((u[2:,1:-1]+u[1:-1,1:-1])*(v[1:-1,1:-1]+v[1:-1,2:])-(u[2:,:-2]+u[1:-1,:-2])*(v[1:-1,1:-1]+v[1:-1,:-2]))/(4*dx)-\
                 ((v[1:-1,1:-1]+[v[2:,1:-1]])**2-(v[1:-1,1:-1]+v[:-2,1:-1])**2)/(4*dy)
        
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [7]:
def Fv2(u,v,dx,dy):
    F=numpy.zeros((numpy.shape(u)))
    F[1:-1,1:-1]=(v[1:-1,2:]-2*v[1:-1,1:-1]+v[1:-1,:-2])/dx**2+(v[2:,1:-1]-2*v[1:-1,1:-1]+v[:-2,1:-1])/dy**2
        
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [8]:
def Fp2(p,dx,dy):
    F=numpy.zeros((numpy.shape(p)))
    F[1:-1,1:-1]=-(p[2:,1:-1]-p[1:-1,1:-1])/dy
        
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [9]:
@jit
def poisson(f,res_target,omega):
    nj,ni=numpy.shape(f)-numpy.array((1,1))
    dx=2*pi/(ni-1)
    dy=2*pi/(nj-1)
    x=numpy.linspace(-dx/2,2*pi+dx/2,ni+1)
    y=numpy.linspace(-dy/2,2*pi+dy/2,nj+1)
    X,Y=numpy.meshgrid(x,y)
    
    p=numpy.zeros(numpy.shape(f))
    res=numpy.zeros(numpy.shape(f))
    
    res_max=1e3
    n=0
    while res_max>res_target:
        
        for j in range(1,nj):
            for i in range(1,ni):
                p[j,i]=omega/(2*(dx**2+dy**2))*((p[j,i-1]+p[j,i+1])*dy**2+\
                       (p[j-1,i]+p[j+1,i])*dx**2-f[j,i]*dx**2*dy**2)+\
                       (1-omega)*p[j,i]
        p[0,:]=p[1,:]
        p[-1,:]=p[-2,:]
        p[:,0]=p[:,1]
        p[:,-1]=p[:,-2]
        n+=1
        
        for j in range(1,nj):
            for i in range(1,ni):
                res[j,i]=numpy.abs(f[j,i]-(p[j,i-1]-2*p[j,i]+p[j,i+1])/dx**2-\
                                  (p[j-1,i]-2*p[j,i]+p[j+1,i])/dy**2)
        res_max=numpy.max(res)
    
    return p

In [10]:
@jit
def getuv(w1,w2,p,dx,dy,dt):
    u=w1+Fp1(p,dx,dy)*dt
    v=w2+Fp2(p,dx,dy)*dt
    
    u[0,:]=-u[1,:]
    u[-1,:]=-u[-2,:]
    u[:,0]=u[:,-2]
    u[:,-1]=u[:,1]

    v[0,:]=v[-2,:]
    v[-1,:]=v[1,:]
    v[:,0]=-v[:,1]
    v[:,-1]=-v[:,-2]
    return u,v

In [11]:
@jit
def error(p,p_ex):
    nj,ni=numpy.shape(p)-numpy.array((1,1))
    err=numpy.zeros(numpy.shape(p))
    for j in range(nj+1):
        for i in range(ni+1):
            err[j,i]=numpy.abs(p[j,i]-p_ex[j,i])
    err_max=numpy.max(err)
    return err_max

In [12]:
@jit
def RK3(ni,nj,dt):
    T=2.
    nt=int(T/dt)
    t=0.
    dx=2.*pi/(ni-1)
    dy=2.*pi/(nj-1)
    x=numpy.linspace(-dx/2,2*pi+dx/2,ni+1)
    y=numpy.linspace(-dy/2,2*pi+dy/2,nj+1)
    xu=numpy.linspace(0,2*pi+dx,ni+1)
    yu=numpy.linspace(-dy/2,2*pi+dy/2,nj+1)
    xv=numpy.linspace(-dx/2,2*pi+dx/2,ni+1)
    yv=numpy.linspace(0,2*pi+dy,nj+1)

    X,Y=numpy.meshgrid(x,y)
    Xu,Yu=numpy.meshgrid(xu,yu)
    Xv,Yv=numpy.meshgrid(xv,yv)

    u0=uex(0,Xu,Yu)
    v0=vex(0,Xv,Yv)
    p0=pex(0,X,Y)
            
    u=u0.copy()
    v=v0.copy()
    p=p0.copy()
    res=numpy.zeros((nj+1,ni+1))
    nt=0
    
    while t<T:
        G1=numpy.zeros((nj+1,ni+1))
        G2=numpy.zeros((nj+1,ni+1))
        w1=numpy.zeros((nj+1,ni+1))
        w2=numpy.zeros((nj+1,ni+1))
        f=numpy.zeros((nj+1,ni+1))
        
        #from t to t+dt/3
        t+=dt/3
        G1=Fc1(u,v,dx,dy)+Fv1(u,v,dx,dy)
        G2=Fc2(u,v,dx,dy)+Fv2(u,v,dx,dy)
        w1=u+dt/3*G1
        w2=v+dt/3*G2
        f[1:-1,1:-1]=3./dt*((w1[1:-1,1:-1]-w1[1:-1,:-2])/dx+(w2[1:-1,1:-1]-w2[:-2,1:-1])/dy)
        p=poisson(f,1e-9,1.99)
        u,v=getuv(w1,w2,p,dx,dy,dt/3)
        
        #from t+dt/3 to t+3dt/4
        t+=5./12*dt
        G1=-5./9*G1+Fc1(u,v,dx,dy)+Fv1(u,v,dx,dy)
        G2=-5./9*G2+Fc2(u,v,dx,dy)+Fv2(u,v,dx,dy)
        w1=u+dt*15./16*G1
        w2=v+dt*15./16*G2
        f[1:-1,1:-1]=12./5/dt*((w1[1:-1,1:-1]-w1[1:-1,:-2])/dx+(w2[1:-1,1:-1]-w2[:-2,1:-1])/dy)        
        p=poisson(f,1e-9,1.99)
        u,v=getuv(w1,w2,p,dx,dy,5.*dt/12)
        
        #from t+3dt/4 to t+dt
        t+=dt/4
        G1=-153./128*G1+Fc1(u,v,dx,dy)+Fv1(u,v,dx,dy)
        G2=-153./128*G2+Fc2(u,v,dx,dy)+Fv2(u,v,dx,dy)
        w1=u+dt*8./15*G1
        w2=v+dt*8./15*G2
        f[1:-1,1:-1]=4./dt*((w1[1:-1,1:-1]-w1[1:-1,:-2])/dx+(w2[1:-1,1:-1]-w2[:-2,1:-1])/dy)
        p=poisson(f,1e-9,1.99)
        u,v=getuv(w1,w2,p,dx,dy,dt/4)
        
        if (nt)%100==0:
            print(error(u,uex(t,Xu,Yu)),error(v,vex(t,Xv,Yv)),error(p,pex(t,X,Y)))
        nt+=1
    
    A=numpy.array([u,v,p])
    return A

In [13]:
def grid_p(ni,nj):
    dx=2*pi/(ni-1)
    dy=2*pi/(nj-1)
    x=numpy.linspace(-dx/2,2*pi+dx/2,ni+1)
    y=numpy.linspace(-dy/2,2*pi+dy/2,nj+1)
    X,Y=numpy.meshgrid(x,y)
    return X,Y

def grid_u(ni,nj):
    dx=2*pi/(ni-1)
    dy=2*pi/(nj-1)
    xu=numpy.linspace(0,2*pi+dx,ni+1)
    yu=numpy.linspace(-dy/2,2*pi+dy/2,nj+1)
    X,Y=numpy.meshgrid(xu,yu)
    return X,Y

def grid_v(ni,nj):
    dx=2*pi/(ni-1)
    dy=2*pi/(nj-1)
    xv=numpy.linspace(-dx/2,2*pi+dx/2,ni+1)
    yv=numpy.linspace(0,2*pi+dy,nj+1)
    X,Y=numpy.meshgrid(xv,yv)
    return X,Y

In [14]:
X,Y=grid_p(41,41)
Xu,Yu=grid_u(41,41)
Xv,Yv=grid_v(41,41)

In [15]:
A1_1=RK3(41,41,0.004)
u_err=error(A1_1[0],uex(2,Xu,Yu))
v_err=error(A1_1[1],vex(2,Xv,Yv))
p_err=error(A1_1[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0.000549384788127 0.000549384788103 0.0924408260144
0.000736830029856 0.000736830029849 0.0416753817525
0.000741172492211 0.000741172492242 0.0188304518598
0.000662702745418 0.000662702745388 0.00850809762589
0.000555507015169 0.000555507015191 0.0038441014879
0.000447024764707 0.000447024764721 0.0017367926355
0.000349735072423 0.000349735072418 0.000784678808115
0.000268035286469 0.000268035286479 0.000354508615455
0.000202211270217 0.000202211270217 0.000160159486419
0.000150668940752 0.000150668940756 7.23551789399e-05
u error=0.000150668940751518
v error=0.00015066894075564663
p error=7.235517893993597e-05


In [16]:
A1_2=RK3(41,41,0.002)
u_err=error(A1_2[0],uex(2,Xu,Yu))
v_err=error(A1_2[1],vex(2,Xv,Yv))
p_err=error(A1_2[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0.000335444162892 0.000335444162908 0.138459375799
0.000549389774941 0.000549389774929 0.0927059696744
0.000674842110327 0.000674842110334 0.0620714733361
0.000736836720889 0.000736836720885 0.0415599891707
0.000754243593486 0.000754243593488 0.0278264505708
0.000741179225445 0.000741179225461 0.0187001304361
0.000708109458737 0.00070810945875 0.0125700667858
0.000662708768247 0.000662708768232 0.00844944454063
0.000610528046562 0.000610528046545 0.00567958195698
0.000555512065869 0.000555512065858 0.00381770379426
0.000500399162305 0.000500399162305 0.00256617191282
0.000447028830726 0.000447028830734 0.0017249119651
0.000396577476029 0.000396577476035 0.00115943338862
0.00034973825482 0.000349738254823 0.000779331735672
0.000306857525727 0.000306857525724 0.000523837547139
0.000268037726451 0.000268037726456 0.00035210208711
0.000233214362745 0.000233214362744 0.00023666732519
0.00020221311174 0.00020221311174 0.000159076391685
0.000174791717414 0.000174791717414 0.000106922944828
0.

In [17]:
A1_3=RK3(41,41,0.001)
u_err=error(A1_3[0],uex(2,Xu,Yu))
v_err=error(A1_3[1],vex(2,Xv,Yv))
p_err=error(A1_3[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0.000185342735223 0.000185342735214 0.16944897866
0.000335444542119 0.000335444542127 0.13865405826
0.000455330935022 0.000455330935029 0.113455609305
0.000549390396159 0.000549390396154 0.0928365763977
0.000621450078734 0.000621450078739 0.0759647331216
0.000674842873563 0.000674842873567 0.0621590935344
0.000712466797034 0.000712466797036 0.050862427551
0.000736837554409 0.000736837554408 0.041618770979
0.000750135039244 0.000750135039245 0.0340550226064
0.000754244446874 0.000754244446875 0.0278658855553
0.000750792605738 0.000750792605728 0.0228015452214
0.000741180064218 0.000741180064226 0.0186575864846
0.000726609411793 0.0007266094118 0.0152791582312
0.000708110260246 0.000708110260252 0.0125269998935
0.000686561264278 0.000686561264285 0.0102705599473
0.000662709518511 0.000662709518503 0.00842055223221
0.000637187630716 0.000637187630708 0.00690377187137
0.000610528737889 0.00061052873788 0.00566019895858
0.000583179701868 0.00058317970186 0.00464062388133
0.000555512695027 0

In [18]:
X,Y=grid_p(21,21)
Xu,Yu=grid_u(21,21)
Xv,Yv=grid_v(21,21)

In [19]:
A2_1=RK3(21,21,0.004)
u_err=error(A2_1[0],uex(2,Xu,Yu))
v_err=error(A2_1[1],vex(2,Xv,Yv))
p_err=error(A2_1[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0.00217451873385 0.00217451873392 0.0987192729154
0.00292003446903 0.00292003446909 0.0440175885923
0.00294086070898 0.00294086070892 0.0196246727907
0.00263274501284 0.00263274501277 0.00896202554481
0.00220960583445 0.00220960583452 0.00411069954339
0.00178029691556 0.0017802969155 0.00188495873915
0.00139455614438 0.00139455614445 0.000864109250033
0.0010701018481 0.00107010184816 0.000396022710889
0.000808304858649 0.000808304858585 0.000181451220193
0.000603018480129 0.000603018480042 8.31173423871e-05
u error=0.0006030184801286871
v error=0.0006030184800419232
p error=8.311734238710479e-05


In [20]:
A2_2=RK3(21,21,0.002)
u_err=error(A2_2[0],uex(2,Xu,Yu))
v_err=error(A2_2[1],vex(2,Xv,Yv))
p_err=error(A2_2[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0.0013268959044 0.00132689590443 0.148181095314
0.00217452356563 0.00217452356566 0.0989532156628
0.00267271557306 0.0026727155731 0.066077763291
0.00292004096798 0.00292004096801 0.0441233967288
0.00299086299063 0.0029908629906 0.0294625677572
0.00294086726495 0.00294086726498 0.0196725279344
0.00281138270321 0.00281138270324 0.0131455600099
0.00263275089156 0.00263275089153 0.00890433833994
0.0024269467148 0.00242694671484 0.00603103599878
0.00220961077645 0.00220961077648 0.00408460865207
0.00199162059834 0.00199162059837 0.00276616317487
0.00178030090385 0.00178030090382 0.00187315829476
0.00158035210038 0.00158035210042 0.00126835558609
0.00139455927364 0.00139455927368 0.000858772121622
0.00122433068829 0.00122433068826 0.000581414296254
0.00107010425324 0.00107010425327 0.000393608820343
0.00093165207903 0.000931652079067 0.000266450012522
0.000808306678312 0.00080830667828 0.000180359460196
0.000699127179023 0.000699127178986 0.000122077275111
0.000603019839804 0.00060301983984

In [21]:
A2_3=RK3(21,21,0.001)
u_err=error(A2_3[0],uex(2,Xu,Yu))
v_err=error(A2_3[1],vex(2,Xv,Yv))
p_err=error(A2_3[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0.000732923049388 0.000732923049373 0.181539065571
0.00132689627139 0.00132689627141 0.14835246255
0.00180167667174 0.00180167667173 0.121231796928
0.00217452416755 0.00217452416757 0.0990684637363
0.00246049747071 0.00246049747073 0.0809564399232
0.0026727163135 0.00267271631351 0.0661552700483
0.00282259373937 0.00282259373939 0.0540598139766
0.00292004177759 0.00292004177761 0.0441755216548
0.00297365346051 0.00297365346052 0.0360982251635
0.00299086382055 0.00299086382053 0.0294976228714
0.0029780922164 0.00297809221639 0.024103781315
0.00294086808167 0.00294086808168 0.0196961032383
0.00288394195976 0.00288394195977 0.0160943114707
0.0028113834846 0.00281138348461 0.0131510803617
0.0027266677846 0.00272666778458 0.0107845281268
0.00263275162388 0.00263275162387 0.00887591922526
0.00253214045042 0.0025321404504 0.00730495286778
0.00242694739044 0.00242694739046 0.00601192354463
0.00231894511393 0.00231894511391 0.00494767886629
0.00220961139208 0.0022096113921 0.00407175512498
0.00

In [22]:
X,Y=grid_p(81,81)
Xu,Yu=grid_u(81,81)
Xv,Yv=grid_v(81,81)

In [23]:
A3_3=RK3(81,81,0.001)
u_err=error(A3_3[0],uex(2,Xu,Yu))
v_err=error(A3_3[1],vex(2,Xv,Yv))
p_err=error(A3_3[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

4.64681834235e-05 4.64681834241e-05 0.14565665056
8.40944631761e-05 8.40944631765e-05 0.119231491854
0.00011414065847 0.000114140658471 0.0976004036637
0.000137708523665 0.000137708523666 0.0798936433546
0.000155758783848 0.000155758783846 0.0653992572564
0.000169127997171 0.000169127997171 0.0535579339541
0.000178543485182 0.000178543485182 0.0438666700845
0.000184636546113 0.000184636546113 0.0359290283551
0.000187954142726 0.000187954142726 0.0294276937559
0.000188969235362 0.000188969235362 0.0241027687667
0.000188089912191 0.000188089912191 0.0197413838064
0.000185667452016 0.000185667452016 0.01616918764
0.000182003440063 0.000182003440063 0.0132433775757
0.000177356044021 0.00017735604402 0.0108469908233
0.000171945545666 0.000171945545665 0.00888422882081
0.000165959212951 0.000165959212951 0.00727662760913
0.000159555587948 0.000159555587948 0.00595992117119
0.000152868257713 0.000152868257712 0.00488147233892
0.000146009167655 0.000146009167656 0.0039981685749
0.0001390715303

In [ ]:
pyplot.figure(figsize=(14,5))
pyplot.subplot(121)
pyplot.contour(Xu,Yu,A2_2[0],10)
pyplot.colorbar()
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')
pyplot.title('$u$ velocity')

pyplot.subplot(122)
pyplot.contour(Xv,Yv,A2_2[1],10)
pyplot.colorbar()
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')
pyplot.title('$v$ velocity');

In [ ]:
pyplot.figure(figsize=(6.5,5))
pyplot.contour(X,Y,A2_2[2],10)
pyplot.colorbar()
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')